In [33]:
import os 
import numpy as np
import json
from math import sqrt
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt
import cv2
import yaml

In [104]:
jsonexample = json.load(open("c/WIN_20230130_08_31_47_Pro.json"))
shape = jsonexample["shapes"]
flatlist = []
labelslist = []
with open("labels.yaml") as file:
    labellist = yaml.load(file, Loader=yaml.FullLoader)

labelcount = 0
normalizedXY = []
for index, label in enumerate(shape):
    labelidx = labellist.index(label["label"])
    with open("testLabel.txt", "w") as file:
        file.write('\n')
        file.write(str(labelidx))
        file.write(' ')
        for point in label["points"]:
            file.write(str(point[0]/jsonexample["imageWidth"], point[1]/jsonexample["imageHeight"]))
            file.write(' ')
        file.write('\n')


TypeError: str() argument 'encoding' must be str, not float

In [ ]:

def min_bounding_rect(points):
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.
    hull_points = points[ConvexHull(points).vertices]
    
    edges = np.zeros((len(hull_points)-1,2))
    edges = hull_points[1:] - hull_points[:-1]
    
    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])
    
    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)
    
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles - pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
    rotations = rotations.reshape((-1,2,2))
    
    rot_points = np.dot(rotations, hull_points.T)
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)
    
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)
    
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    coords = (x1,y1,x2,y2)
    return coords
    
rect = min_bounding_rect(flatlist)

In [ ]:
def xyxy2xywh(x):
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = (x[:, 0] + x[:, 2]) / 2
    y[:, 1] = (x[:, 1] + x[:, 3]) / 2

In [3]:
x1 = abs(int(rect[0]))
y1 = abs(int(rect[1]))
x2 = abs(int(rect[2]))
y2 = abs(int(rect[3]))
testimg = cv2.imread("c/WIN_20230130_08_31_47_Pro.jpg", 1)
rectimg = cv2.rectangle(testimg,(x1,y1), (x2,y2), (255,0,0),3)
cv2.imwrite("c/testrect.png", rectimg)


True